# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

#### Import the necessary libraries

In [47]:
from nltk.tag import pos_tag
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline
import string
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Andres Lojan
[nltk_data]     Yepez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andres Lojan
[nltk_data]     Yepez\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [2]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [3]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [36]:
def tuples_maker(input_sigs, output_labels):
    sample_data=[]
    
    for a,b in zip(input_sigs, output_labels):
        inner_data=[]
        for c,d in zip(a, b):
            element=(c,d)
            inner_data.append(element) #sample_data
        sample_data.append(inner_data)
    
    return sample_data

In [52]:
whole_data=tuples_maker(input_sigs, output_labels)
print(whole_data)

[[('for', 'FOR'), ('5', 'Duration'), ('to', 'TO'), ('6', 'DurationMax'), ('days', 'DurationUnit')], [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')], [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')], [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')], [('every', 'EVERY'), ('day', 'Period')], [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')], [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')], [('every', 'EVERY'), ('1', 'Period'), ('to', 'TO'), ('2', 'PeriodMax'), ('months', 'PeriodUnit')], [('every', 'EVERY'), ('2', 'Period'), ('to', 'TO'), ('6', 'PeriodMax'), ('weeks', 'PeriodUnit')], [('every', 'EVERY'), ('4', 'Period'), ('to', 'TO'), ('6', 'PeriodMax'), ('days', 'PeriodUnit')], [('take', 'Method'), ('two', 'Qty'), ('to', 'TO'), ('four', 'Qty'), ('tabs', 'Form')], [('take', 'Method'), ('2', 'Qty'), ('to', 'TO'), ('4', 'Qty'), ('tabs', 'Form')], [('take', 'Method'), ('3', 'Qty'), ('tabs', 'Form'), ('orally', 'PO'), ('bid', 'BID'), ('

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [49]:
def triples_maker(whole_data):
    demo = []
    sample_data = []
    for x in whole_data:
        for i, j in x:
            temp = nltk.word_tokenize(i)
            pos = nltk.pos_tag(temp)
            temp2 = (pos[0][0], pos[0][1], j)
            demo.append(temp2) 
        sample_data.append(demo)
        demo = []
    return sample_data

In [53]:
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [55]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [56]:
from sklearn.model_selection import train_test_split as tts

def get_features(doc):
    return [token_to_features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [get_features(doc) for doc in sample_data]

y = [get_labels(doc) for doc in sample_data]

xtrain, xtest, ytrain, ytest = tts(X, y, test_size=0.2)

### Training the CRF model with the features extracted using the feature extractor method

In [57]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(xtrain, ytrain):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({ 'c1': 0.2, 'c2': 0.01, 'max_iterations': 150, 'feature.possible_transitions': True})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1846
Seconds required: 0.005

L-BFGS optimization
c1: 0.200000
c2: 0.010000
num_memories: 6
max_iterations: 150
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 591.056350
Feature norm: 1.000000
Error norm: 115.393011
Active features: 921
Line search trials: 1
Line search step: 0.006588
Seconds required for this iteration: 0.003

***** Iteration #2 *****
Loss: 383.255442
Feature norm: 5.411266
Error norm: 146.253950
Active features: 1083
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 269.050547
Feature norm: 6.618254
Error norm: 58.823267
Active features: 1070
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0

### Predicting the test data with the built model

In [58]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
ypred = [tagger.tag(xseq) for xseq in xtest]

In [59]:
print(f"F-Score = {metrics.flat_f1_score(ytest, ypred, average='weighted')}")
print(metrics.flat_classification_report(ytest, ypred,  digits=3))

F-Score = 0.7697619047619049
              precision    recall  f1-score   support

       AFTER      1.000     1.000     1.000         1
         AND      0.000     0.000     0.000         1
          AT      0.500     1.000     0.667         1
      BEFORE      1.000     1.000     1.000         1
         BID      0.000     0.000     0.000         1
    Duration      0.750     0.750     0.750         4
DurationUnit      0.750     0.750     0.750         4
       EVERY      1.000     1.000     1.000         1
        FOOD      1.000     1.000     1.000         1
         FOR      0.750     1.000     0.857         3
        Form      0.857     0.857     0.857         7
   Frequency      0.500     0.500     0.500         2
      Method      1.000     0.667     0.800         3
          PO      0.000     0.000     0.000         1
      Period      1.000     1.000     1.000         3
  PeriodUnit      1.000     1.000     1.000         3
        Q46H      0.000     0.000     0.000         

C:\Users\Andres Lojan Yepez\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\Andres Lojan Yepez\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Andres Lojan Yepez\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Andres Lojan Yepez\anaconda3\lib\s

### Putting all the prediction logic inside a predict method

In [60]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    demo = []
    sample_data = []
    sig = [sig]
    for x in sig:
        temp = nltk.word_tokenize(x)
        pos = nltk.pos_tag(temp)
        for y in range(len(pos)):
            temp2 = (pos[y][0], pos[y][1], pos[y][1]) 
            demo.append(temp2) 
            sample_data.append(demo)
            demo = []
    data = [get_features(doc) for doc in sample_data]
    tagger.open('crf.model')
    predictions = [tagger.tag(xseq) for xseq in data]
    
    return predictions

### Sample predictions

In [61]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
print(predictions)

[['Method'], ['Qty'], ['Form'], ['EVERY'], ['Qty'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['DurationUnit']]


In [62]:
predictions = predict("2 capsu for 10 day at bed")
print(predictions)

[['Qty'], ['WHEN'], ['FOR'], ['Qty'], ['PeriodUnit'], ['AT'], ['WHEN']]


In [63]:
predictions = predict("2 capsu for 10 days at bed")
print(predictions)

[['Qty'], ['WHEN'], ['FOR'], ['Qty'], ['DurationUnit'], ['AT'], ['WHEN']]


In [64]:
predictions = predict("5 days 2 tabs at bed")
print(predictions)

[['Qty'], ['DurationUnit'], ['Qty'], ['Form'], ['AT'], ['WHEN']]


In [65]:
predictions = predict("3 tabs qid x 10 weeks")
print(predictions)

[['Qty'], ['Form'], ['QID'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [66]:
predictions = predict("x 30 days")
print(predictions)

[['PeriodUnit'], ['Qty'], ['DurationUnit']]


In [67]:
predictions = predict("x 20 months")
print(predictions)

[['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [69]:
predictions = predict("take 2 tabs po tid for 10 days")
print(predictions)

[['Method'], ['Qty'], ['Form'], ['DAILY'], ['TID'], ['FOR'], ['Qty'], ['DurationUnit']]


In [71]:
predictions = predict("take 2 capsules po every 6 hours")
print(predictions)

[['Method'], ['Qty'], ['PeriodUnit'], ['PO'], ['EVERY'], ['Qty'], ['PeriodUnit']]


In [72]:
predictions = predict("inject 2 units pu tid")
print(predictions)

[['DAILY'], ['Qty'], ['PeriodUnit'], ['PeriodUnit'], ['TID']]


In [73]:
predictions = predict("swallow 3 caps tid by mouth")
print(predictions)

[['Method'], ['Qty'], ['PeriodUnit'], ['TID'], ['BY'], ['WHEN']]


In [75]:
predictions = predict("inject 3 units orally")
print(predictions)

[['DAILY'], ['Qty'], ['PeriodUnit'], ['PO']]


In [76]:
predictions = predict("orally take 3 tabs tid")
print(predictions)

[['PO'], ['PeriodUnit'], ['Qty'], ['Form'], ['TID']]


In [77]:
predictions = predict("by mouth take three caps")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [107]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
print(predictions)

[['Method'], ['Qty'], ['Form'], ['PO'], ['Qty'], ['TIMES'], ['Period'], ['PeriodUnit'], ['FOR'], ['Qty'], ['DurationUnit'], ['AT'], ['WHEN']]


In [108]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")
print(predictions)

[['Method'], ['Qty'], ['Form'], ['PO'], ['BID'], ['FOR'], ['Qty'], ['DurationUnit'], ['AT'], ['WHEN']]


In [106]:
predictions = predict("take 3 tabs bid orally at bed")
print(predictions)

[['Method'], ['Qty'], ['Form'], ['BID'], ['PO'], ['AT'], ['WHEN']]


In [105]:
predictions = predict("take 10 capsules by mouth qid")
print(predictions)

[['Method'], ['Qty'], ['PeriodUnit'], ['BY'], ['WHEN'], ['QID']]


In [104]:
predictions = predict("inject 10 units orally qid x 3 months")
print(predictions)

[['DAILY'], ['Qty'], ['PeriodUnit'], ['PO'], ['QID'], ['DAILY'], ['Qty'], ['PeriodUnit']]


In [103]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [79]:
prediction = predict("Amoxcicillin QID 30 tablets")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [80]:
prediction = predict("take 3 tabs TID for 90 days with food")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [81]:
prediction = predict("with food take 3 tablets per day for 90 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [82]:
prediction = predict("with food take 3 tablets per week for 90 weeks")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [83]:
prediction = predict("take 2-4 tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [84]:
prediction = predict("take 2 to 4 tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [85]:
prediction = predict("take two to four tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [86]:
prediction = predict("take 2-4 tabs for 8 to 9 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [87]:
prediction = predict("take 20 tabs every 6 to 8 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [88]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [89]:
prediction = predict("take 2 tabs every 2 to 10 weeks")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [90]:
prediction = predict("take 2 tabs every 4 to 6 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [91]:
prediction = predict("take 2 tabs every 2 to 10 months")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [92]:
prediction = predict("every 60 mins")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [93]:
prediction = predict("every 10 mins")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [94]:
prediction = predict("every two to four months")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [95]:
prediction = predict("take 2 tabs every 3 to 4 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [96]:
prediction = predict("every 3 to 4 days take 20 tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [97]:
prediction = predict("once in every 3 days take 3 tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [98]:
prediction = predict("take 3 tabs once in every 3 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [99]:
prediction = predict("orally take 20 tabs every 4-6 weeks")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [100]:
prediction = predict("10 tabs x 2 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [101]:
prediction = predict("3 capsule x 15 days")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]


In [102]:
prediction = predict("10 tabs")
print(predictions)

[['BY'], ['PeriodUnit'], ['PeriodUnit'], ['Qty'], ['PeriodUnit']]
